# Import libraries

In [2]:
import pandas as pd
import numpy as np
import os
import config
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score, accuracy_score
from sklearn.metrics import confusion_matrix
from tqdm import tqdm

# Load data

In [3]:
X = pd.read_csv(os.path.join(config.CLEAN_DIR, "taxonomic_features.csv"), index_col=[0, 1])
y = pd.read_csv(os.path.join(config.CLEAN_DIR, "metadata.csv"), index_col=[0, 1]).iloc[:, [0]]
display(X.head())
display(y.head())

k__Archaea  k__Archaea|p__Euryarchaeota  \
Study_ID Sample Accession                                            
GMHI-10  SAMN03283239        0.000000                     0.000000   
         SAMN03283266        0.000000                     0.000000   
         SAMN03283281        0.009764                     0.009764   
         SAMN03283294        0.000000                     0.000000   
         SAMN03283288        0.011865                     0.011865   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria  \
Study_ID Sample Accession                                                   
GMHI-10  SAMN03283239                                            0.000000   
         SAMN03283266                                            0.000000   
         SAMN03283281                                            0.009764   
         SAMN03283294                                            0.000000   
         SAMN03283288                                            0.011865   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales  \
Study_ID Sample Accession                                                                         
GMHI-10  SAMN03283239                                               0.000000                      
         SAMN03283266                                               0.000000                      
         SAMN03283281                                               0.009764                      
         SAMN03283294                                               0.000000                      
         SAMN03283288                                               0.011865                      

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae  \
Study_ID Sample Accession                                                                                                
GMHI-10  SAMN03283239                                               0.000000                                             
         SAMN03283266                                               0.000000                                             
         SAMN03283281                                               0.009764                                             
         SAMN03283294                                               0.000000                                             
         SAMN03283288                                               0.011865                                             

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter  \
Study_ID Sample Accession                                                                                                                      
GMHI-10  SAMN03283239                                               0.000000                                                                   
         SAMN03283266                                               0.000000                                                                   
         SAMN03283281                                               0.009764                                                                   
         SAMN03283294                                               0.000000                                                                   
         SAMN03283288                                               0.011865                                                                   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter|s__Methanobrevibacter_smithii  \
Study_ID Sample Accession                                                                                                                                                    
GMHI-10  SAMN03283239                                               0.000000              

PHENOTYPE:Healthy_Nonhealthy
Study_ID Sample Accession                              
GMHI-10  SAMN03283239                              True
         SAMN03283266                              True
         SAMN03283281                              True
         SAMN03283294                              True
         SAMN03283288                              True

# Generate predictions in LOOCV

In [4]:
np.random.seed(42)

In [6]:
leave = LeaveOneOut()
logit_prediction_list = []
    
# instantiate the logistic regression classifier (gmhi2)
gmhi2 = LogisticRegression(random_state=42, penalty="l1", solver="liblinear", C=config.REGULARIZATION, class_weight="balanced")

# generate logit predictions in LOOCV
GMHI2_scores_cv = cross_val_predict(gmhi2, X > config.PRESENCE_CUTOFF, y.values.flatten(), method="decision_function",
    cv=leave, verbose=2, n_jobs=-1
)

GMHI2_scores_cv = pd.DataFrame(GMHI2_scores_cv, index=y.index, columns=["GMHI2_cv"])
logit_prediction_list.append(GMHI2_scores_cv)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:   30.7s
[Parallel(n_jobs=-1)]: Done 333 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 616 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 981 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 1426 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 1953 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 2560 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done 3249 tasks      | elapsed:  9.1min
[Parallel(n_jobs=-1)]: Done 4018 tasks      | elapsed: 11.3min
[Parallel(n_jobs=-1)]: Done 4869 tasks      | elapsed: 13.6min
[Parallel(n_jobs=-1)]: Done 5800 tasks      | elapsed: 16.3min
[Parallel(n_jobs=-1)]: Done 6813 tasks      | elapsed: 19.5min
[Parallel(n_jobs=-1)]: Done 7906 tasks      | elapsed: 22.6min
[Parallel(n_jobs=-1)]: Done 8204 out of 8204 |

In [16]:
def evaluate_performance(cutoff):
    balanced_accuracies = [balanced_accuracy_score(y[(abs(lo) >= cutoff).values], lo[(abs(lo) >= cutoff).values] > 0) for lo in logit_prediction_list]
    mean_acc = np.mean(balanced_accuracies)
    print("Mean acc:", mean_acc)
    
    mat_sum = np.zeros([2 , 2])

    for lo in logit_prediction_list:
        idx = (abs(lo) >= cutoff).values
        y_curr = y[idx]
        lo_curr = lo[idx]
        mat = confusion_matrix(y_curr, lo_curr > 0)
        mat_sum += mat
    tn, fp, fn, tp = mat_sum.ravel()
    df = pd.DataFrame(mat_sum, columns=["Predicted Nonhealthy", "Predicted Healthy"], index=["Actual Nonhealthy", "Actual healthy"])
    display(df)
    print("Total samples evaluated:", mat_sum.sum())
    print("Percentage of possible:", mat_sum.sum() * 100 / X.shape[0])

In [17]:
evaluate_performance(0)

Mean acc: 0.7853729368745368


,Predicted Nonhealthy,Predicted Healthy
Actual Nonhealthy,2086.0,571.0
Actual healthy,1189.0,4358.0


Total samples evaluated: 8204.0
Percentage of possible: 100.0


In [18]:
evaluate_performance(0.1)

Mean acc: 0.7998351704418848


,Predicted Nonhealthy,Predicted Healthy
Actual Nonhealthy,2032.0,502.0
Actual healthy,1073.0,4233.0


Total samples evaluated: 7840.0
Percentage of possible: 95.56313993174061


In [19]:
evaluate_performance(1)

Mean acc: 0.8999224861200386


,Predicted Nonhealthy,Predicted Healthy
Actual Nonhealthy,1291.0,142.0
Actual healthy,333.0,2962.0


Total samples evaluated: 4728.0
Percentage of possible: 57.63042418332521
